# ETL with Spark (Local)

In [1]:
from pyspark.sql import SparkSession
# from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType, DateType, TimestampType

# import pyspark.sql.functions as F

In [2]:
import pandas as pd
import glob

In [3]:
p = glob.glob("data/*.json")

In [4]:
p

['data/github_events_02.json', 'data/github_events_01.json']

In [6]:
data = "github_events_01.json"

In [7]:
data_2 = "github_events_02.json"

In [8]:
spark = SparkSession.builder \
    .appName("ETL") \
    .getOrCreate()

In [9]:
data_folder = "data"

In [10]:
data = spark.read.option("multiline", "true").json(data_folder)

In [62]:
# data = spark.read.option("multiline", "true").json(data_2)

In [11]:
data.show()

+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+--------------------+
|               actor|          created_at|         id|                 org|             payload|public|                repo|                type|
+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+--------------------+
|{https://avatars....|2022-08-17T15:52:40Z|23487963576|{https://avatars....|{started, NULL, N...|  true|{6296790, spring-...|          WatchEvent|
|{https://avatars....|2022-08-17T15:52:40Z|23487963624|                NULL|{NULL, NULL, NULL...|  true|{525860969, gurra...|         CreateEvent|
|{https://avatars....|2022-08-17T15:52:40Z|23487963529|                NULL|{NULL, e80c84c7bb...|  true|{350706029, afbel...|           PushEvent|
|{https://avatars....|2022-08-17T15:52:40Z|23487963558|{https://avatars....|{created, NULL, {...|  true|{226399669, CM

In [12]:
data.printSchema()

root
 |-- actor: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- display_login: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- org: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- payload: struct (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- before: string (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- author_association: string (nullable = true)
 |    |    |-- body: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- html_url: string (nullable 

In [13]:
data.select("id", "type").show()

+-----------+--------------------+
|         id|                type|
+-----------+--------------------+
|23487963576|          WatchEvent|
|23487963624|         CreateEvent|
|23487963529|           PushEvent|
|23487963558|   IssueCommentEvent|
|23487963581|    PullRequestEvent|
|23487963532|           PushEvent|
|23487963524|           PushEvent|
|23487963526|           PushEvent|
|23487963492|           PushEvent|
|23487963504|         DeleteEvent|
|23487963536|PullRequestReview...|
|23487963495|         CreateEvent|
|23487963522|           PushEvent|
|23487963444|           PushEvent|
|23487963462|    PullRequestEvent|
|23487963480|         IssuesEvent|
|23487963457|           PushEvent|
|23487963413|           PushEvent|
|23487963429|           PushEvent|
|23487963448|           PushEvent|
+-----------+--------------------+
only showing top 20 rows



In [14]:
data.createOrReplaceTempView("staging_events")

In [15]:
table = spark.sql("""
    select
        *
        
    from
        staging_events
""").show()

+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+--------------------+
|               actor|          created_at|         id|                 org|             payload|public|                repo|                type|
+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+--------------------+
|{https://avatars....|2022-08-17T15:52:40Z|23487963576|{https://avatars....|{started, NULL, N...|  true|{6296790, spring-...|          WatchEvent|
|{https://avatars....|2022-08-17T15:52:40Z|23487963624|                NULL|{NULL, NULL, NULL...|  true|{525860969, gurra...|         CreateEvent|
|{https://avatars....|2022-08-17T15:52:40Z|23487963529|                NULL|{NULL, e80c84c7bb...|  true|{350706029, afbel...|           PushEvent|
|{https://avatars....|2022-08-17T15:52:40Z|23487963558|{https://avatars....|{created, NULL, {...|  true|{226399669, CM

In [16]:
table = spark.sql("""
    select
        id
        , type
        , created_at
        , to_date(created_at) as date
        , year(created_at) as year
        , actor.login
        , actor.url as actor_url
        , repo.name
        , repo.url as repo_url
        
    from
        staging_events
""")

In [17]:
table.show()

+-----------+--------------------+--------------------+----------+----+--------------+--------------------+--------------------+--------------------+
|         id|                type|          created_at|      date|year|         login|           actor_url|                name|            repo_url|
+-----------+--------------------+--------------------+----------+----+--------------+--------------------+--------------------+--------------------+
|23487963576|          WatchEvent|2022-08-17T15:52:40Z|2022-08-17|2022|    evilgaoshu|https://api.githu...|spring-projects/s...|https://api.githu...|
|23487963624|         CreateEvent|2022-08-17T15:52:40Z|2022-08-17|2022|      gurram47|https://api.githu...|gurram47/AP201100...|https://api.githu...|
|23487963529|           PushEvent|2022-08-17T15:52:40Z|2022-08-17|2022|    afbeltranr|https://api.githu...| afbeltranr/Agrilab2|https://api.githu...|
|23487963558|   IssueCommentEvent|2022-08-17T15:52:40Z|2022-08-17|2022|      karla-vm|https://api.gi

In [18]:
output_csv = "output_csv"
output_parquet = "output_parquet"

In [27]:
table.write.partitionBy("year").mode("overwrite").csv(output_csv)

In [28]:
table.write.partitionBy("year").mode("overwrite").parquet(output_parquet)

In [19]:
table = spark.sql("""
    select
        id
        , type
        , created_at
        , day(created_at) as day
        , month(created_at) as month
        , year(created_at) as year
        , date(created_at) as date
    from
        staging_events
""")

In [20]:
table.show()

+-----------+--------------------+--------------------+---+-----+----+----------+
|         id|                type|          created_at|day|month|year|      date|
+-----------+--------------------+--------------------+---+-----+----+----------+
|23487963576|          WatchEvent|2022-08-17T15:52:40Z| 17|    8|2022|2022-08-17|
|23487963624|         CreateEvent|2022-08-17T15:52:40Z| 17|    8|2022|2022-08-17|
|23487963529|           PushEvent|2022-08-17T15:52:40Z| 17|    8|2022|2022-08-17|
|23487963558|   IssueCommentEvent|2022-08-17T15:52:40Z| 17|    8|2022|2022-08-17|
|23487963581|    PullRequestEvent|2022-08-17T15:52:40Z| 17|    8|2022|2022-08-17|
|23487963532|           PushEvent|2022-08-17T15:52:40Z| 17|    8|2022|2022-08-17|
|23487963524|           PushEvent|2022-08-17T15:52:40Z| 17|    8|2022|2022-08-17|
|23487963526|           PushEvent|2022-08-17T15:52:40Z| 17|    8|2022|2022-08-17|
|23487963492|           PushEvent|2022-08-17T15:52:40Z| 17|    8|2022|2022-08-17|
|23487963504|   

In [50]:
destination = "events"

In [52]:
table.write.partitionBy("year", "month", "day").mode("overwrite").csv(destination)

In [55]:
table.write.partitionBy("date").mode("overwrite").csv(destination)

In [37]:
table = spark.sql("""
    select
        actor.login
        , id as event_id
        , actor.url as actor_url
    from
        staging_events
""")
destination = "actors"
table.write.mode("overwrite").csv(destination)

In [38]:
table = spark.sql("""
    select
        repo.name
        , id as event_id
        , repo.url as repo_url
        
    from
        staging_events
""")
destination = "repos"
table.write.mode("overwrite").csv(destination)